### Analisi nelle subreddit

Per valutare se la propensione la verbosità di un interazione utente post varia in funzione della size di un post, analizziamo:
- il numero di utenti univoci in un thread
- la distribuzione della lunghezza (in termini di numero di commenti) delle interazioni

Dividiamo quindi i thread in bins in base alla size, assicurandoci che per ogni bin ci siano almeno 1000 thread dentro, e su questi calcoliamo la distribuzione della lunghezza delle interazioni utente post facendo affidamento al parametro locator per vedere se la verbosità aumenta. (Tagliando le code, mettendo un upper clip per interazioni con più di 10 commenti).

Questo è stato fatto sia dividendo le conversazioni in base a gli anni in cui avvengono sia in base ai subreddit a cui appartengono.

In [2]:
# Jupyter Notebook: reddit_analysis.ipynb

# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from EXP_package.temporal_funcions import *
import os

# Set platform
platform = 'Reddit'

# Define data and output directories
data_directory = '/media/jacoponudo/Elements/Reddit/'
output_directory = '/home/jacoponudo/Documents/Size_effects/PLT/7_temporal/'

# Define files to process
files_to_process = [
    'reddit_climatechange.csv',
    'reddit_conspiracy.csv',
    'reddit_news.csv',
    'reddit_science.csv'
]

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Call the function to plot for individual files
plot_location_parameters(data_directory, output_directory, files_to_process, platform)

# Process raw data and plot
raw_data_files = ['reddit_raw_data_unified.parquet']
combined_data = process_raw_data(data_directory, raw_data_files)
plot_location_parameters_by_year(combined_data, output_directory, platform)


/home/jacoponudo/Documents/Size_effects/EXP/EXP_package/temporal_funcions.py:26: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(data_directory, filename)).drop_duplicates(subset='comment_code')
/home/jacoponudo/Documents/Size_effects/EXP/EXP_package/temporal_funcions.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  location_params = data.groupby('size_bin')['interaction_len'].apply(
/home/jacoponudo/Documents/Size_effects/EXP/EXP_package/temporal_funcions.py:26: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(os.path.join(data_directory, filename)).drop_duplicates(subset='comment_code')
/home/jacoponudo/Documents/Size_effects/EXP/EXP_pa

In [ ]:
# Jupyter Notebook: reddit_analysis.ipynb

# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from EXP_package.temporal_funcions import *
import os

# Set platform
platform = 'Voat'

# Define data and output directories
data_directory = '/media/jacoponudo/Elements/Reddit/'
output_directory = '/home/jacoponudo/Documents/Size_effects/PLT/7_temporal/'

# Define files to process
files_to_process = [
    'reddit_climatechange.csv',
    'reddit_conspiracy.csv',
    'reddit_news.csv',
    'reddit_science.csv'
]

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Call the function to plot for individual files
plot_location_parameters(data_directory, output_directory, files_to_process, platform)

# Process raw data and plot
raw_data_files = ['reddit_raw_data_unified.parquet']
combined_data = process_raw_data(data_directory, raw_data_files)
plot_location_parameters_by_year(combined_data, output_directory, platform)
